In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.signal import correlate as cr
#adf test
from statsmodels.tsa.stattools import adfuller
#cross correlation
from scipy import signal

#derivate
from sympy import diff, Symbol

Importando dados

In [2]:
#importando dados - INDICADORES
dados_trimestrais = pd.read_csv("C:\\workspace_mestrado\\dados_cvm_trimestrais\\base_dados_tri.csv", sep=";")
dados_anuais = pd.read_csv("C:\\workspace_mestrado\\dados_cvm_anuais\\base_dados.csv", sep=";")

Tratamento de dados

In [3]:
#filtro inicial
def filtro_inicial(dados : pd.DataFrame,  empresa: str) -> pd.DataFrame:

    cd_cvm= dados[dados['DENOM_CIA'] == empresa]['CD_CVM'].unique()[0]
    regex = r"^\d{1}\.\d{2}$"
    
    #filtrar linhas da base_dados_tri que contenham o formato regex
    dados_anuais_regex = dados[dados['CD_CONTA'].str.contains(regex)]
    dados_anuais_filtrados = dados_anuais_regex[['CD_CVM','CD_CONTA', 'ESCALA_MOEDA', 'DT_INI_EXERC','DT_FIM_EXERC','DS_CONTA', 'VL_CONTA', 'ST_CONTA_FIXA']].drop_duplicates()

    dados_anuais_empresa = dados_anuais_filtrados[dados_anuais_filtrados['CD_CVM'] == cd_cvm]
    dados_anuais_empresa['DT_INI_EXERC'] = pd.to_datetime(dados_anuais_empresa['DT_INI_EXERC'])
    dados_anuais_empresa['DT_FIM_EXERC'] = pd.to_datetime(dados_anuais_empresa['DT_FIM_EXERC'])

        #pegar o CD_CONTA a partir do nome da empresa, para capturar seus dados mesmo que tenha mudado de nome ao longo de sua história

    return dados_anuais_empresa

In [4]:
def compilado_trimestral(dados_trimestrais: pd.DataFrame, dados_anuais: pd.DataFrame):
    dados_trimestrais_copia = dados_trimestrais.copy()
    dados_anuais_copia = dados_anuais.copy()
    dados_3tri = dados_trimestrais_copia[((dados_trimestrais_copia['DT_FIM_EXERC'] - dados_trimestrais_copia['DT_INI_EXERC']) < datetime.timedelta(days=100)) &
                                         ((dados_trimestrais_copia['DT_FIM_EXERC'] - dados_trimestrais_copia['DT_INI_EXERC']) > datetime.timedelta(days=31))]
    #print(dados_3tri)

    new_data_df = pd.DataFrame(columns=['CD_CVM', 'CD_CONTA', 'ESCALA_MOEDA', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'DS_CONTA', 'VL_CONTA', 'ST_CONTA_FIXA'])
    for ano in range(2011, 2023):
        print({ano})
        resultado_4t = dados_3tri[(dados_3tri['DT_FIM_EXERC'].dt.year == ano)]
        print("Filtro Ano Executado")
        print(resultado_4t['CD_CONTA'])
        
        for cd_conta in resultado_4t['CD_CONTA']:

            print(f"Filtro Conta Executado - {cd_conta}")
            #somando valores dos 3 trimestres
            value_condicao_ano = sum(resultado_4t[(resultado_4t['CD_CONTA'] == cd_conta)]['VL_CONTA'])
            #transformar value_condicao_ano em float
            total_3tri = float(value_condicao_ano)
            print(f"{cd_conta} - Somados 3 trimestres: {total_3tri}")
                
            total_anual = dados_anuais_copia[(dados_anuais_copia['CD_CONTA'] == cd_conta) & (dados_anuais_copia['DT_FIM_EXERC'].dt.year == ano)]['VL_CONTA'].iloc[0]

            value_4t = total_anual - total_3tri
            print(f"value_4t: {value_4t}, total_anual: {total_anual}, total_3tris: {total_3tri}")

            #ADICIONANDO 4 TRIMESTRE NO DATAFRAME
            new_data = {'CD_CVM': [dados_3tri['CD_CVM'].iloc[0]], 'CD_CONTA': [cd_conta], 'ESCALA_MOEDA':  [resultado_4t[(resultado_4t['CD_CONTA'] == cd_conta)]['ESCALA_MOEDA'].iloc[0]],
                        'DT_INI_EXERC': [datetime.datetime(ano, 10, 1)], 'DT_FIM_EXERC': [datetime.datetime(ano, 12, 31)], 'DS_CONTA': [resultado_4t[(resultado_4t['CD_CONTA'] == cd_conta)]['DS_CONTA'].iloc[0]],
                        'VL_CONTA': [value_4t], 'ST_CONTA_FIXA': [resultado_4t[(resultado_4t['CD_CONTA'] == cd_conta)]['ST_CONTA_FIXA'].iloc[0]]}
            
            #transformar new_data em dataframe sendo que cd_vm, cd_conta, escala_moeda, ds_conta e st_conta_fixa são colunas
            new_data_df = pd.concat([pd.DataFrame(new_data_df), pd.DataFrame(new_data)])
            
            #juntar dataframe new_data_df com dados_3tri
        
            #print(f"{cd_conta} - {ano} - Executado")
            print(new_data_df)

            #encerrar todos os loops
        #encerrar todos os loops
        

    
    #exportar dados_3tri para csv
    new_data_df.to_csv("C:\\workspace_mestrado\\dados_cvm_trimestrais\\teste_4tri_v2.csv", sep=";")

    dados_trimestrais_completos = pd.concat([dados_3tri, new_data_df], ignore_index=True)
    return dados_trimestrais_completos

            
            
        #somar todas as linhas 
        
    

In [5]:
#executando funções
empresa = "TRANSMISSORA ALIANÇA DE ENERGIA ELÉTRICA S.A."
dados_anuais_empresa = filtro_inicial(dados_anuais, empresa)
dados_trimestrais_empresa = filtro_inicial(dados_trimestrais, empresa)

C:\Users\giova\AppData\Local\Temp\ipykernel_35212\697196913.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_anuais_empresa['DT_INI_EXERC'] = pd.to_datetime(dados_anuais_empresa['DT_INI_EXERC'])
C:\Users\giova\AppData\Local\Temp\ipykernel_35212\697196913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_anuais_empresa['DT_FIM_EXERC'] = pd.to_datetime(dados_anuais_empresa['DT_FIM_EXERC'])
C:\Users\giova\AppData\Local\Temp\ipykernel_35212\697196913.py:12: SettingWithCopyWarning: 
A value is

In [6]:
dados_trimestrais_completos = compilado_trimestral(dados_trimestrais_empresa, dados_anuais_empresa)
dados_trimestrais_completos = dados_trimestrais_completos.drop_duplicates()
dados_trimestrais_completos = dados_trimestrais_completos.sort_values(by=['DT_FIM_EXERC'])

C:\Users\giova\AppData\Local\Temp\ipykernel_35212\1423182868.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data_df = pd.concat([pd.DataFrame(new_data_df), pd.DataFrame(new_data)])


{2011}
Filtro Ano Executado
19805    3.01
19806    3.02
19812    3.03
19813    3.04
19824    3.05
19825    3.06
19828    3.07
19829    3.08
19832    3.09
19833    3.10
19836    3.11
19839    3.99
19847    3.01
19849    3.02
19861    3.03
19863    3.04
19885    3.05
19887    3.06
19893    3.07
19895    3.08
19901    3.09
19903    3.10
19909    3.11
19915    3.99
19929    3.01
19931    3.02
19943    3.03
19945    3.04
19967    3.05
19969    3.06
19975    3.07
19977    3.08
19983    3.09
19985    3.10
19991    3.11
19997    3.99
Name: CD_CONTA, dtype: object
Filtro Conta Executado - 3.01
3.01 - Somados 3 trimestres: 798372.0
value_4t: 198872.0, total_anual: 997244.0, total_3tris: 798372.0
  CD_CVM CD_CONTA ESCALA_MOEDA DT_INI_EXERC DT_FIM_EXERC  \
0  20257     3.01          MIL   2011-10-01   2011-12-31   

                                 DS_CONTA  VL_CONTA ST_CONTA_FIXA  
0  Receita de Venda de Bens e/ou Serviços  198872.0             S  
Filtro Conta Executado - 3.02
3.02 - Somados 3 t

In [7]:
#concatenar dados_4tri com dados_trimestrais da empresa
dados_trimestrais_completos.to_csv(f"C:\\workspace_mestrado\\dados_cvm_trimestrais\\dados_trimestrais_completos_{empresa}.csv", sep=";")